In [1]:
import csv
import sqlite3
import warnings
from pathlib import Path

from PIL import Image
from tqdm import tqdm

In [2]:
DATA_DIR = Path(".") / "data" / "sernec"
OCR_DIR = DATA_DIR / "sernec_2023-05-18_ocr"
LABEL_DIR = DATA_DIR / "sernec_2023-05-18_labels"
DB = DATA_DIR / "sernec.sqlite"
LABEL_IDS = DATA_DIR / "sernec_2023-05-18_label_ids.csv"

In [3]:
sql = """
    select path, coreid, labels.*, ocr_text
    from ocr_texts
    join labels using (label_id)
    join sheets using (sheet_id)
    where ocr_set = 'sernec_2023-05-18'
    """
with sqlite3.connect(DB) as cxn, open(LABEL_IDS, "w") as csv_out:
    writer = csv.writer(csv_out)
    writer.writerow(["old_id", "new_id"])
    
    cxn.row_factory = sqlite3.Row
    
    for row in tqdm(cxn.execute(sql)):
        name = (
            f"{row['coreid']}_{row['class'].lower()}_"
            f"{row['label_left']}_{row['label_top']}_"
            f"{row['label_right']}_{row['label_bottom']}.txt"
        )
        with open(OCR_DIR / name, "w") as f:
            f.write(row["ocr_text"])

        with warnings.catch_warnings():  # Turn off EXIF warnings
            warnings.filterwarnings("ignore", category=UserWarning)
            sheet = Image.open(row["path"])

        image = sheet.crop(
            (
                row["label_left"],
                row["label_top"],
                row["label_right"],
                row["label_bottom"],
            )
        )
        
        name = (LABEL_DIR / name).with_suffix(".jpg")
        image.save(name, "JPEG")
        
        writer.writerow([row["label_id"], name.stem])

1224it [00:00, 48067.75it/s]
